# Import Libraries

In [145]:
# Load libraries
import numpy as np
from matplotlib import pyplot
from pycaret.regression import *
from pycaret.regression import load_model, predict_model
from pandas import set_option
set_option('display.max_rows', 500)
set_option('display.max_columns', 500)
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from datetime import datetime
import pyodbc
import os
import optuna
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

import warnings
warnings.filterwarnings("ignore")

# Connect to SQL Server

In [2]:
pyodbc.drivers()

['SQL Server',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client RDA 11.0']

In [3]:
connection = pyodbc.connect(
"Driver={ODBC Driver 17 for SQL Server};"
"Server=DESKTOP-ROTIMI\SQLEXPRESS;"
"Database=Rossmann;"
"Trusted_Connection=yes;")

# load dataset

#### Extracting data from SQL Server

In [4]:
# View data from SQL Server
train = pd.read_sql("Select * from [dbo].[train_cleaned]",connection)
test = pd.read_sql("Select * from [dbo].[test_cleaned]",connection)
store = pd.read_sql("Select * from [dbo].[store_cleaned]",connection)


# Train Table

#### View Data

In [5]:
# head
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,5,5,2015-07-31,4822,559,1,1,0,1
4,6,5,2015-07-31,5651,589,1,1,0,1


In [6]:
# head
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


In [7]:
# head
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_Int,Assortment_Int,PromoInterval_int
0,1,c,a,1270,9,2008,0,0,0,period 0,3,1,0
1,2,a,a,570,11,2007,1,13,2010,Period 1,1,1,1
2,3,a,a,14130,12,2006,1,14,2011,Period 1,1,1,1
3,4,c,c,620,9,2009,0,0,0,period 0,3,3,0
4,5,a,a,29910,4,2015,0,0,0,period 0,1,1,0


#### shape of the data

In [8]:
# shape
print(train.shape)


(814299, 9)


In [9]:
# shape
print(test.shape)

(41088, 8)


In [10]:
# shape
print(store.shape)


(1115, 13)


#### types of the data column

In [11]:
# types
print(train.dtypes)


Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday      int64
SchoolHoliday     int64
dtype: object


In [12]:
# types
print(test.dtypes)

Id                int64
Store             int64
DayOfWeek         int64
Date             object
Open              int64
Promo             int64
StateHoliday      int64
SchoolHoliday     int64
dtype: object


In [13]:
# types
print(store.dtypes)

Store                         int64
StoreType                    object
Assortment                   object
CompetitionDistance           int64
CompetitionOpenSinceMonth     int64
CompetitionOpenSinceYear      int64
Promo2                        int64
Promo2SinceWeek               int64
Promo2SinceYear               int64
PromoInterval                object
StoreType_Int                 int64
Assortment_Int                int64
PromoInterval_int             int64
dtype: object


#### data descriptions

In [14]:
# descriptions
train.describe()


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,814299.000000,814299.000000,814299.000000,814299.000000,814299.000000,814299.000000,814299.000000,814299.000000
mean,557.386423,3.546504,6578.110456,718.145233,0.999000,0.435819,0.000996,0.192362
std,323.195128,1.714786,2429.153669,306.621660,0.031601,0.495864,0.031543,0.394156
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,277.000000,2.000000,4798.000000,513.000000,1.000000,0.000000,0.000000,0.000000
50%,557.000000,4.000000,6250.000000,663.000000,1.000000,0.000000,0.000000,0.000000
75%,840.000000,5.000000,8078.000000,861.000000,1.000000,1.000000,0.000000,0.000000
max,1115.000000,7.000000,13610.000000,4256.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
# descriptions
test.describe()


,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854361,0.395833,0.004381,0.443487
std,11861.228267,320.274496,2.015481,0.352748,0.489035,0.066044,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,0.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
# descriptions
store.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,StoreType_Int,Assortment_Int,PromoInterval_int
count,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000
mean,558.00000,5404.898655,4.930942,1370.939013,0.512108,12.083408,1030.239462,2.217040,1.928251,0.818834
std,322.01708,7652.849306,4.284924,935.467654,0.500078,15.542241,1006.038782,1.369544,0.993814,0.976108
min,1.00000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,279.50000,720.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,558.00000,2330.000000,4.000000,2006.000000,1.000000,1.000000,2009.000000,1.000000,1.000000,1.000000
75%,836.50000,6875.000000,9.000000,2011.000000,1.000000,22.000000,2012.000000,4.000000,3.000000,1.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000,4.000000,3.000000,3.000000


#### data descriptions

In [17]:
# descriptions
train.describe(include = 'object')


,Date
count,814299
unique,942
top,2013-12-24
freq,1114


In [18]:
# descriptions
test.describe(include = 'object')


,Date
count,41088
unique,48
top,2015-09-17
freq,856


In [19]:
# descriptions
store.describe(include = 'object')

,StoreType,Assortment,PromoInterval
count,1115,1115,1115
unique,4,3,4
top,a,a,period 0
freq,602,593,544


#### Check for missing entries

In [20]:
# Check for missing entries
total = train.isnull().sum().sort_values(ascending=False)
percent=(train.isnull().sum()*100/len(train)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Store,0,0.0
DayOfWeek,0,0.0
Date,0,0.0
Sales,0,0.0
Customers,0,0.0
Open,0,0.0
Promo,0,0.0
StateHoliday,0,0.0
SchoolHoliday,0,0.0


In [21]:
# Check for missing entries
total = test.isnull().sum().sort_values(ascending=False)
percent=(test.isnull().sum()*100/len(test)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Id,0,0.0
Store,0,0.0
DayOfWeek,0,0.0
Date,0,0.0
Open,0,0.0
Promo,0,0.0
StateHoliday,0,0.0
SchoolHoliday,0,0.0


In [22]:
# Check for missing entries
total = store.isnull().sum().sort_values(ascending=False)
percent=(store.isnull().sum()*100/len(store)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Store,0,0.0
StoreType,0,0.0
Assortment,0,0.0
CompetitionDistance,0,0.0
CompetitionOpenSinceMonth,0,0.0
CompetitionOpenSinceYear,0,0.0
Promo2,0,0.0
Promo2SinceWeek,0,0.0
Promo2SinceYear,0,0.0
PromoInterval,0,0.0


#### Check for duplicate data

In [23]:
# locate rows of duplicate data

# calculate duplicates
dups = train.duplicated()
# report if there are any duplicates
print(dups.any())


False


In [24]:
# locate rows of duplicate data

# calculate duplicates
dups = test.duplicated()
# report if there are any duplicates
print(dups.any())


False


In [25]:
# locate rows of duplicate data

# calculate duplicates
dups = store.duplicated()
# report if there are any duplicates
print(dups.any())

False


# Feature Engineering

#### conversion to datetime

In [26]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

#### Create time-based features

In [27]:
# Create time-based features (train)
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train["WeekOfYear"] = train["Date"].dt.isocalendar().week
train["IsWeekend"] = train["DayOfWeek"].apply(lambda x: 1 if x >= 5 else 0)  # 1 = Weekend
train["Season"] = train["Month"].map({1: "winter", 2: "winter", 3: "spring", 4: "spring", 5: "spring", 6: "summer", 7: "summer", 8: "summer", 9: "fall", 10: "fall", 11: "fall", 12: "winter"})


# Create time-based features (test)
test["Year"] = test["Date"].dt.year
test["Month"] = test["Date"].dt.month
test["WeekOfYear"] = test["Date"].dt.isocalendar().week
test["IsWeekend"] = test["DayOfWeek"].apply(lambda x: 1 if x >= 5 else 0)  # 1 = Weekend
test["Season"] = test["Month"].map({1: "winter", 2: "winter", 3: "spring", 4: "spring", 5: "spring", 6: "summer", 7: "summer", 8: "summer", 9: "fall", 10: "fall", 11: "fall", 12: "winter"})


#### Extract text-based details

In [28]:
# Extract text-based details (train)
train['Month_01'] = train['Date'].dt.strftime('%B')  # Full month name
train['Day'] = train['Date'].dt.strftime('%d')    # Day as text
train['Day of Week'] = train['Date'].dt.strftime('%A')  # Full weekday name

# Extract text-based details (test)
test['Month_01'] = test['Date'].dt.strftime('%B')  # Full month name
test['Day'] = test['Date'].dt.strftime('%d')    # Day as text
test['Day of Week'] = test['Date'].dt.strftime('%A')  # Full weekday name

#### Lag Features (Past Sales Trends)

In [29]:
# Lag Features (Past Sales Trends : train)
train["Sales_Lag_7"] = train.groupby("Store")["Sales"].shift(7)
train["Sales_Lag_7"]


0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
814294     5775.0
814295    12577.0
814296     7659.0
814297     4260.0
814298     8057.0
Name: Sales_Lag_7, Length: 814299, dtype: float64

In [30]:
# Lag Features (Past Sales Trends : train)
train["Sales_Lag_30"] = train.groupby("Store")["Sales"].shift(30)
train["Sales_Lag_30"]

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
814294    11235.0
814295    11807.0
814296     9244.0
814297     5405.0
814298     9390.0
Name: Sales_Lag_30, Length: 814299, dtype: float64

#### Rolling Mean for Trend Analysis

In [31]:
# Rolling Mean for Trend Analysis : train
train["Sales_MA_7"] = train.groupby("Store")["Sales"].transform(lambda x: x.rolling(7).mean())
train["Sales_MA_7"]

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
814294     8260.285714
814295    11817.714286
814296     6977.428571
814297     4114.285714
814298     6896.571429
Name: Sales_MA_7, Length: 814299, dtype: float64

In [32]:
# Lag Features and Rolling Mean : test
test['Sales_Lag_7'] = train['Sales_Lag_7'].median()
test['Sales_Lag_30'] = train['Sales_Lag_30'].median()
test['Sales_MA_7'] = train['Sales_MA_7'].median()

In [33]:
train.dropna(inplace = True)

#### Encoding category to numeric

In [34]:
# Encoding category to numeric
le = LabelEncoder()
train['Season_01'] = le.fit_transform(train['Season'])
test['Season_01'] = le.fit_transform(test['Season'])

#### change types

In [35]:
train['Season_01'] = train['Season_01'].astype('int32')
test['Season_01'] = test['Season_01'].astype('int32')

#### Merge table

In [36]:
# Merge train and store data
train_store = train.merge(store, on="Store", how="left")

# Merge test and store data
test_store = test.merge(store, on="Store", how="left")

In [37]:
test_store.head(2)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,WeekOfYear,IsWeekend,Season,Month_01,Day,Day of Week,Sales_Lag_7,Sales_Lag_30,Sales_MA_7,Season_01,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StoreType_Int,Assortment_Int,PromoInterval_int
0,1,1,4,2015-09-17,1,1,0,0,2015,9,38,0,fall,September,17,Thursday,6249.0,6266.0,6386.571429,0,c,a,1270,9,2008,0,0,0,period 0,3,1,0
1,2,3,4,2015-09-17,1,1,0,0,2015,9,38,0,fall,September,17,Thursday,6249.0,6266.0,6386.571429,0,a,a,14130,12,2006,1,14,2011,Period 1,1,1,1


#### drop categories column

In [38]:
# drop categories for train columnvalues
train_store.drop(columns=['Day of Week','Season','Month_01','StoreType','Assortment','PromoInterval'],inplace = True)

# drop categories for test column values
test_store.drop(columns=['Day of Week','Season','Month_01','StoreType','Assortment','PromoInterval'],inplace = True)

#### convert type


In [39]:
# convert type in train_store column
train_store['Day'] = train_store['Day'].astype('int32')
train_store['Year'] = train_store['Day'].astype('int32')
train_store['Month'] = train_store['Month'].astype('int32')
train_store['WeekOfYear'] = train_store['WeekOfYear'].astype('int32')

# convert type in test_store column
test_store['Day'] = test_store['Day'].astype('int32')
test_store['Year'] = test_store['Day'].astype('int32')
test_store['Month'] = test_store['Month'].astype('int32')
test_store['WeekOfYear'] = test_store['WeekOfYear'].astype('int32')


#### Correlation

In [40]:
train_store_corr = train_store.select_dtypes(include=['int64']).copy()
# correlation
train_store_corr.corr(method='pearson')

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,IsWeekend,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,StoreType_Int,Assortment_Int,PromoInterval_int
Store,1.000000,0.000352,0.002248,0.037611,0.028132,-0.000210,-0.000583,-0.000601,-0.000075,-0.025651,-0.033335,-0.010631,0.010042,0.010737,0.010073,-0.019315,0.004611,0.000936
DayOfWeek,0.000352,1.000000,-0.175263,-0.102236,-0.011420,-0.277937,-0.002509,-0.134700,0.835157,0.007305,-0.000314,-0.000313,-0.004504,-0.002708,-0.004503,-0.004977,0.003126,-0.005293
Sales,0.002248,-0.175263,1.000000,0.768742,0.085596,0.402816,-0.003935,0.032707,-0.109643,-0.008263,-0.015544,0.001455,-0.093943,-0.030873,-0.093970,0.028014,0.104123,-0.104625
Customers,0.037611,-0.102236,0.768742,1.000000,0.074251,0.193126,0.022682,0.023363,-0.067040,-0.140841,-0.013326,0.007767,-0.186853,-0.121457,-0.186815,-0.197815,-0.017998,-0.169044
Open,0.028132,-0.011420,0.085596,0.074251,1.000000,0.005137,-0.066987,-0.004840,-0.007050,0.004371,-0.009379,-0.010113,0.010853,0.011176,0.010852,-0.007068,0.008703,0.010644
Promo,-0.000210,-0.277937,0.402816,0.193126,0.005137,1.000000,0.003303,0.017739,-0.272838,-0.001926,-0.000107,-0.001278,0.002752,-0.000395,0.002752,0.005824,-0.003547,0.004382
StateHoliday,-0.000583,-0.002509,-0.003935,0.022682,-0.066987,0.003303,1.000000,0.013828,-0.013555,0.013614,-0.000957,-0.003972,-0.007244,-0.008822,-0.007239,-0.003605,-0.001361,-0.006875
SchoolHoliday,-0.000601,-0.134700,0.032707,0.023363,-0.004840,0.017739,0.013828,1.000000,-0.127385,-0.003966,0.001568,0.001285,-0.003699,-0.003631,-0.003702,0.003136,-0.003717,-0.004223
IsWeekend,-0.000075,0.835157,-0.109643,-0.067040,-0.007050,-0.272838,-0.013555,-0.127385,1.000000,0.004966,-0.000370,-0.000391,-0.001935,-0.000915,-0.001935,-0.003407,0.001205,-0.002779
CompetitionDistance,-0.025651,0.007305,-0.008263,-0.140841,0.004371,-0.001926,0.013614,-0.003966,0.004966,1.000000,-0.032719,-0.003553,-0.152167,-0.132110,-0.152228,0.071140,0.152920,-0.130552


#### Correlation Matrix

In [41]:
train_store_corr.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'IsWeekend', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'StoreType_Int', 'Assortment_Int',
       'PromoInterval_int'],
      dtype='object')

- **Note**
   1. **IsWeekend** and **DayOfWeek** have a *high Positive correlation* of is **0.84** which means they share the same information
   2. **Customers** and **Sales** have a positive correlation of is **0.77**
   3. The *correlation* between **Sales** and **Promo** is **0.40** 
   4. **CompetitionOpenSinceMonth** and **CompetitionOpenSinceYear** have a *high positive correlation* of **0.79** <br>
      They share similar information (One of the column should be retained).
   5. The *correlation* between **Promo2** and **PromoInterval_int** is **0.82** which is *highly correlative*
   6. The *correlation* between **Promo2** and **Promo2SinceWeek** is **0.76** which is *highly correlative*
   7. The *correlation* between **Promo2sinceYear** and **Promo2SinceWeek** is **0.76** which is *highly correlative*                                                                                      
   8. The *correlation* between **Promo2** and **Promo2sinceYear** is *highly correlative* at **1.0**
   9. The *correlation* between **Promo2sinceYear** and **PromoInterval_int** is **0.82** 
  10. The *correlation* between **WeekOfYear** and **Month** is **0.96** <br> <br>

- **Action Taken**
   1. **DayOfWeek**,**Customers**,**CompetitionOpenSinceYear**, will be drop
   2. Since **Promo2** is *highly correlative* with **Promo2sinceYear** of **1.0** and *positively correlated* with *other three columns* then drop <br>
      **PromoInterval_int**,**Promo2SinceWeek** and **Promo2sinceYear**.
   3. **WeekOfYear** should be drop       

 

# Visualization

## Scatter Plot
#### Sales Over Customers

In [ ]:
plt.figure(figsize = (10,4))
plt.title('Scatter Plot Of Sales By Customers')
plt.scatter(data = train_store ,x = 'Customers' ,y = 'Sales',color = 'green')
plt.xlabel('Customers')
plt.ylabel('Sales')

## Line Plot
#### Sales Over Time

In [ ]:
plt.figure(figsize = (12,6))
sns.lineplot(data = train_store,x = 'Date' , y ='Sales',ci = None )
plt.title('Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.xticks(rotation = 45)
plt.show()

## Line Plot
#### Customers Over Time

In [ ]:
plt.figure(figsize = (15,4))
plt.title('Scatter Plot Of Customers By Date')
sns.lineplot(data = train_store,x = 'Date' , y ='Customers',color = 'red' )
plt.xlabel('Date')
plt.ylabel('Customers')

## Box Plot
#### Sales by Day of the Week

In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(data = train_store,x = 'DayOfWeek' , y ='Sales' )
plt.title('Sales Distribution by Day of the Week')
plt.xlabel('Day of the week (1=Monday,7=Sunday)')
plt.show()

## Histogram
#### Sales Distribution

In [ ]:
plt.figure(figsize = (10,6))
sns.histplot(train_store['Sales'] , bins = 50 , kde = True)
plt.title('Sales Distribution')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.show()

## Bar Chart
#### Average Sales Per Store Type

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(data = train_store , x = 'StoreType_Int' , y = 'Sales' ,estimator = sum)
plt.title('Total Sales by StoreType')
plt.xlabel('StoreType')
plt.ylabel('Total Sales')
plt.show()

## Bar Chart
#### Average Sales by Assortment

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(data = train_store , x = 'Assortment_Int' , y = 'Sales' ,estimator = sum)
plt.title('Total Sales by Assortment Type')
plt.xlabel('Assortment')
plt.ylabel('Total Sales')
plt.show()

## Correlation Heatmap
#### Sales versus Other Featrues

#### Action Taken from Observation

In [ ]:
DayOfWeek,Customers,CompetitionOpenSinceYear,WeekOfYear will be drop
Since Promo2 is highly correlative with Promo2sinceYear of 1.0 and positively correlated with other three columns then drop
PromoInterval_int,Promo2SinceWeek and Promo2sinceYear.

#### drop unecessary columns

In [44]:
#removal of redundant columns
train_store.drop(columns = ['DayOfWeek','Customers','CompetitionOpenSinceYear','PromoInterval_int','Promo2SinceWeek',
                            'Promo2SinceYear','WeekOfYear'],inplace = True)

#removal of redundant columns
test_store.drop(columns = ['DayOfWeek','CompetitionOpenSinceYear','PromoInterval_int','Promo2SinceWeek',
                            'Promo2SinceYear','WeekOfYear'],inplace = True)


#### VIF : Checking for Multicollinearity

In [ ]:
X = train_store_corr[['Store', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month',
       'IsWeekend', 'Day', 'Season_01', 'CompetitionDistance',
       'CompetitionOpenSinceMonth',  'Promo2', 'StoreType_Int', 'Assortment_Int']]
# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display VIF values
print(vif_data)

#### drop columns with high VIF

In [45]:
#removal of columns with high VIF
train_store.drop(columns = ['Open'],inplace = True)
test_store.drop(columns = ['Open'],inplace = True)


#### Check for any high VIF

In [ ]:
X = train_store_corr[['Store', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month',
       'IsWeekend', 'Day', 'Season_01', 'CompetitionDistance',
       'CompetitionOpenSinceMonth',  'Promo2', 'StoreType_Int', 'Assortment_Int']]

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display VIF values
print(vif_data)

#### rename columns

In [46]:
# rename column
train_store.rename(columns ={'StoreType_Int' : 'StoreType', 'Assortment_Int' : 'Assortment','Season_01' : 'Season'},inplace =True)
test_store.rename(columns ={'StoreType_Int' : 'StoreType', 'Assortment_Int' : 'Assortment','Season_01' : 'Season'},inplace =True)

In [47]:
train_store.head(2)

,Store,Date,Sales,Promo,StateHoliday,SchoolHoliday,Year,Month,IsWeekend,Day,Sales_Lag_7,Sales_Lag_30,Sales_MA_7,Season,CompetitionDistance,CompetitionOpenSinceMonth,Promo2,StoreType,Assortment
0,1,2015-07-01,5223,1,0,0,1,7,0,1,3797.0,5263.0,4036.000000,2,1270,9,0,3,1
1,85,2015-07-01,7820,1,0,0,1,7,0,1,6705.0,7791.0,7595.714286,2,1870,10,0,2,1



#### Type conversion

In [50]:
# converting type from 64 to 32 to reduce memory
train_store.select_dtypes(include=['int64']).columns
train_store['Store'] = train_store['Store'].astype('int32')
train_store['Sales'] = train_store['Sales'].astype('int32')
train_store['Promo'] = train_store['Promo'].astype('int32')
train_store['StateHoliday'] = train_store['StateHoliday'].astype('int32')
train_store['SchoolHoliday'] = train_store['SchoolHoliday'].astype('int32')
train_store['IsWeekend'] = train_store['IsWeekend'].astype('int32')
train_store['CompetitionDistance'] = train_store['CompetitionDistance'].astype('int32')
train_store['CompetitionOpenSinceMonth'] = train_store['CompetitionOpenSinceMonth'].astype('int32')
train_store['CompetitionOpenSinceMonth'] = train_store['CompetitionOpenSinceMonth'].astype('int32')
train_store['Promo2'] = train_store['Promo2'].astype('int32')
train_store['StoreType'] = train_store['StoreType'].astype('int32')
train_store['Assortment'] = train_store['Assortment'].astype('int32')
train_store['Sales_Lag_7'] = train_store['Sales_Lag_7'].astype('int32')
train_store['Sales_Lag_30'] = train_store['Sales_Lag_30'].astype('int32')
train_store['Sales_MA_7'] = train_store['Sales_MA_7'].astype('int32')

                                                             


# pycaret: low code Automl libraries


#### Split the data

In [51]:
traindf,testdf =train_test_split(train_store,random_state=2021,test_size=0.2)
traindf.shape,testdf.shape

((624741, 19), (156186, 19))

In [52]:
# Avoid memory overload
n_jobs = min(os.cpu_count(),4)

#### Pycaret SetUp

In [53]:
History=setup(data=traindf,target='Sales',
      normalize=True,train_size=0.20,test_data = testdf,ignore_features = ['Date'],normalize_method = 'minmax',transformation = True,
              fold = 3,use_gpu = False,
              n_jobs = n_jobs,session_id=42
             ,verbose = False)



#### compare_models

In [54]:

best = compare_models(include = ['lightgbm',
                          'xgboost',
                          'catboost'],sort = 'R2',n_select=1)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,587.2102,646515.9312,804.0571,0.8899,0.2700,0.0984,65.4900
xgboost,Extreme Gradient Boosting,598.9179,670993.1149,819.1326,0.8858,0.2714,0.1006,14.1100
lightgbm,Light Gradient Boosting Machine,661.6453,801336.4721,895.1712,0.8636,0.2772,0.1121,14.5200



#### Define Optuna's Objective Function for CatBoost

In [55]:

# Step 4: Define Optuna's Objective Function for CatBoost
def objective(trial):
    """Optimize CatBoost hyperparameters using Optuna."""
    
    # Define hyperparameter search space
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, step=0.01),
        "depth": trial.suggest_int("depth", 4, 10),
        "iterations": trial.suggest_int("iterations", 100, 1000, step=100),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10, step=1),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0, step=0.1),
        "verbose": 0  # Silence CatBoost logging
    }
    
    # Train CatBoostRegressor manually
    model = CatBoostRegressor(**params)
    model.fit(traindf.drop(columns=['Sales']), traindf['Sales'])

    # Predict on validation set
    preds = model.predict(testdf.drop(columns=['Sales']))

    # Calculate R2
    r2 = r2_score(testdf['Sales'], preds)

    return r2  # Minimize R2



#### Run Optuna Optimization

In [59]:
# Step 5: Run Optuna Optimization (20 Trials for Faster Tuning)
study = optuna.create_study(direction="maximize")  # Maximize R2
study.optimize(objective, n_trials=10)  # Adjust `n_trials` for more fine-tuning


[I 2025-02-24 02:48:37,191] A new study created in memory with name: no-name-a4157e5d-01a7-420e-be6f-18005a6a6dce
[I 2025-02-24 02:49:54,251] Trial 0 finished with value: 0.908104410775886 and parameters: {'learning_rate': 0.15000000000000002, 'depth': 8, 'iterations': 600, 'l2_leaf_reg': 3.0, 'bagging_temperature': 1.0}. Best is trial 0 with value: 0.908104410775886.
[I 2025-02-24 02:50:05,372] Trial 1 finished with value: 0.85611902900493 and parameters: {'learning_rate': 0.22, 'depth': 5, 'iterations': 100, 'l2_leaf_reg': 8.0, 'bagging_temperature': 0.1}. Best is trial 0 with value: 0.908104410775886.
[I 2025-02-24 02:50:57,949] Trial 2 finished with value: 0.9024938965040614 and parameters: {'learning_rate': 0.18000000000000002, 'depth': 8, 'iterations': 400, 'l2_leaf_reg': 8.0, 'bagging_temperature': 0.6000000000000001}. Best is trial 0 with value: 0.908104410775886.
[I 2025-02-24 02:51:46,219] Trial 3 finished with value: 0.8308503400482099 and parameters: {'learning_rate': 0.02,



#### Get the Best Hyperparameters from Optuna

In [67]:
# Step 6: Get the Best Hyperparameters from Optuna
best_params = study.best_params
print("✅ Best Hyperparameters Found:", best_params)


✅ Best Hyperparameters Found: {'learning_rate': 0.26, 'depth': 9, 'iterations': 900, 'l2_leaf_reg': 1.0, 'bagging_temperature': 0.30000000000000004}



####  Get the Best R2 from Optuna

In [72]:

# Step 6: Get the Best Hyperparameters from Optuna
best_r2= study.best_value
print("✅ Best Hyperparameters Found:", best_r2)


✅ Best Hyperparameters Found: 0.9265598340424801



#### Train CatBoost Model with Best Hyperparameters

In [68]:

# Step 4: Train CatBoost Model with Best Hyperparameters
final_model = create_model('catboost', **best_params)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,495.2108,479562.3628,692.5044,0.9182,0.2596,0.0822
1,493.7274,471300.1539,686.5130,0.9198,0.2470,0.0815
2,495.0361,474408.0102,688.7728,0.9193,0.2628,0.0816
Mean,494.6581,475090.1756,689.2634,0.9191,0.2565,0.0817
Std,0.6620,3407.3486,2.4704,0.0007,0.0068,0.0003



#### predict on validation set

In [153]:

# Step 5: Predict on `testdf`
predictions = predict_model(final_model, data=testdf)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,451.9596,389888.2613,624.4103,0.9338,0.2491,0.0755


In [75]:

# Step 7: Inspect Predictions to Check Column Names
predictions.head()  # Debugging step: Check if 'Label' exists


,Store,Date,Promo,StateHoliday,SchoolHoliday,Year,Month,IsWeekend,Day,Sales_Lag_7,Sales_Lag_30,Sales_MA_7,Season,CompetitionDistance,CompetitionOpenSinceMonth,Promo2,StoreType,Assortment,Sales,prediction_label
532875,685,2013-10-02,0,0,0,2,10,0,2,4776,4757,8768,0,650,11,1,1,1,7614,7070.930450
700710,860,2013-03-30,0,0,0,30,3,1,30,5055,4173,5583,1,5980,2,0,3,3,9508,8240.750242
15459,990,2015-06-10,0,0,0,10,6,0,10,7465,6569,6308,2,20930,0,0,4,1,4297,5109.360093
429990,57,2014-01-28,0,0,0,28,1,0,28,12907,8433,10605,3,420,6,0,4,3,9212,9202.359812
323972,103,2014-05-26,0,0,0,26,5,0,26,5200,5557,4910,1,5210,5,0,4,3,4921,4284.367015



#### Extract Actual & Predicted Sales

In [77]:
# Step 8: Extract Actual & Predicted Sales
actual_sales = testdf['Sales']  # Actual Sales values


# Ensure the correct column name for predictions
if 'Label' in predictions.columns:
    predicted_sales = predictions['Label']  # PyCaret usually stores predictions here
else:
    predicted_sales = predictions.iloc[:, -1]  # Last column might be predictions

#### Compute R² Score

In [85]:
# Step 9: Compute R² Score
r2 = r2_score(actual_sales, predicted_sales)
mae = mean_absolute_error(actual_sales, predicted_sales)
mse = mean_squared_error(actual_sales, predicted_sales)
rmse = np.sqrt(mean_squared_error(actual_sales, predicted_sales))

#### Print the Score

In [87]:

# Step 10: Print R² Score
print(f"✅ R² Score on Test Data: {r2:.4f}")
print(f"✅ mean_absolute_error Score on Test Data: {mae:.4f}")
print(f"✅ mean_squared_error Score on Test Data: {mse:.4f}")
print(f"✅ Root_mean_squared_error Score on Test Data: {rmse:.4f}")

✅ R² Score on Test Data: 0.9338
✅ mean_absolute_error Score on Test Data: 451.9596
✅ mean_squared_error Score on Test Data: 389888.2613
✅ Root_mean_squared_error Score on Test Data: 624.4103



#### evaluate_model(final_model)

In [99]:

evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


## Unseen Data : Test Data

#### Check for missing column

In [108]:
# Check for missing column
missing_cols = set(traindf.columns) - set(test.columns)
print("Missing Columns:", missing_cols)

Missing Columns: {'CompetitionOpenSinceMonth', 'Promo2', 'Assortment', 'CompetitionDistance', 'Sales', 'StoreType'}


#### fill the missing entries 

In [109]:
# fill the missing entries 
for col in missing_cols:
    test[col] = 0  # Use 0 for numerical, '' for categorical

# Reorder unseen_data to match train_data column order
test = test[traindf.columns]

#### change type 

In [121]:

#### change type to match 
for col in traindf.columns:
    test[col] = test[col].astype(traindf[col].dtype)

####  Finalize the model

In [ ]:

# Step 5: Finalize the model (Train on full dataset)
final_model = finalize_model(final_model)

#### Save Result

In [144]:

save_model(final_model,'Predicted_pycaret_result_sale')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Store', 'Promo', 'StateHoliday',
                                              'SchoolHoliday', 'Year', 'Month',
                                              'IsWeekend', 'Day', 'Sales_Lag_7',
                                              'Sales_Lag_30', 'Sales_MA_7',
                                              'Season', 'CompetitionDistance',
                                              'CompetitionOpenSinceMonth',
                                              'Promo2', 'StoreType',
                                              'Assortment'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('transformation',
                  TransformerWrapp


#### Load Model for prediction

In [146]:
# Load trained model

load_model_result= load_model('Predicted_pycaret_result_sale')

Transformation Pipeline and Model Successfully Loaded


#### view the test predicted result

In [149]:
predicted_model=predict_model(load_model_result,data=test)
predicted_model.head(5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,5951.6408,36249530.0911,6020.7583,0.0000,8.6814,nan


,Store,Date,Promo,StateHoliday,SchoolHoliday,Year,Month,IsWeekend,Day,Sales_Lag_7,Sales_Lag_30,Sales_MA_7,Season,CompetitionDistance,CompetitionOpenSinceMonth,Promo2,StoreType,Assortment,Sales,prediction_label
0,1,2015-09-17,1,0,0,2015,9,0,17,6249,6266,6386,0,0,0,0,0,0,0,7758.700019
1,3,2015-09-17,1,0,0,2015,9,0,17,6249,6266,6386,0,0,0,0,0,0,0,7758.700019
2,7,2015-09-17,1,0,0,2015,9,0,17,6249,6266,6386,0,0,0,0,0,0,0,7677.288572
3,8,2015-09-17,1,0,0,2015,9,0,17,6249,6266,6386,0,0,0,0,0,0,0,7677.288572
4,9,2015-09-17,1,0,0,2015,9,0,17,6249,6266,6386,0,0,0,0,0,0,0,7683.104856


#### export the result csv format

In [151]:

predicted_model[['Store','Date','prediction_label']].sort_values(by = ['Date']).reset_index(drop =True).to_csv('predicted_pycaret_sales.csv')